第三章 评估输入——分类

在本节中，我们将专注于评估输入的任务，这对于确保系统的质量和安全性非常重要。

对于需要处理不同情况下的许多独立指令集的任务，首先对查询类型进行分类，然后根据该分类确定要使用哪些指令会很有好处。

这可以通过定义固定的类别和hard-coding与处理给定类别任务相关的指令来实现。

例如，在构建客户服务助手时，首先对查询类型进行分类，然后根据该分类确定要使用哪些指令可能比较重要。

因此，例如，如果用户要求关闭其帐户，您可能会给出不同的辅助指令，而如果用户询问特定产品，则可能会添加其他产品信息。


## Setup
加载 API_KEY 并封装一个调用 API 的函数

In [9]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

#### 对用户指令进行分类

在这里，我们有我们的系统消息，它是对整个系统的指导，并且我们正在使用这个分隔符——#。

分隔符只是一种分隔指令或输出不同部分的方式，它有助于模型确定不同的部分。

因此，对于这个例子，我们将使用#作为分隔符。

这是一个很好的分隔符，因为它实际上被表示为一个token。

In [3]:
delimiter = "####"

这是我们的系统消息，我们正在以下面的方式询问模型。

In [4]:
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""

In [5]:
# 中文 Prompt
system_message = f"""
你将获得客户服务查询。
每个客户服务查询都将用{delimiter}字符分隔。
将每个查询分类到一个主要类别和一个次要类别中。
以JSON格式提供你的输出，包含以下键：primary和secondary。

主要类别：计费（Billing）、技术支持（Technical Support）、账户管理（Account Management）或一般咨询（General Inquiry）。

计费次要类别：
取消订阅或升级（Unsubscribe or upgrade）
添加付款方式（Add a payment method）
收费解释（Explanation for charge）
争议费用（Dispute a charge）

技术支持次要类别：
常规故障排除（General troubleshooting）
设备兼容性（Device compatibility）
软件更新（Software updates）

账户管理次要类别：
重置密码（Password reset）
更新个人信息（Update personal information）
关闭账户（Close account）
账户安全（Account security）

一般咨询次要类别：
产品信息（Product information）
定价（Pricing）
反馈（Feedback）
与人工对话（Speak to a human）

"""

现在我们来看一个用户消息的例子，我们将使用以下内容。

In [26]:
user_message = f"""\ 
I want you to delete my profile and all of my user data"""

In [6]:
user_message = f"""\ 
我希望你删除我的个人资料和所有用户数据。"""

将这个消息格式化为一个消息列表，系统消息和用户消息使用####"进行分隔。

让我们想一想，作为人类，这句话什么意思："我想让您删除我的个人资料。"

这句话看上去属于"Account Management"类别，也许是属于"Close account"这一项。 

In [7]:
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]

让我们看看模型是如何思考的

模型的分类是"Account Management"作为"primary"，"Close account"作为"secondary"。

请求结构化输出（如JSON）的好处是，您可以轻松地将其读入某个对象中，

例如Python中的字典，或者如果您使用其他语言，则可以使用其他对象作为输入到后续步骤中。

In [10]:
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "账户管理",
    "secondary": "关闭账户"
}


这是另一个用户消息: "告诉我更多关于你们的平板电视"

我们只是有相同的消息列表，模型的响应，然后我们打印它。

结果这里是我们的第二个分类，看起来应该是正确的。

In [31]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}


In [12]:
user_message = f"""\
告诉我更多有关你们的平板电脑的信息"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

以下是针对平板电脑的一般咨询：

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}

如果您有任何特定的问题或需要更详细的信息，请告诉我，我会尽力回答。


所以总的来说，根据客户咨询的分类，我们现在可以提供一套更具体的指令来处理后续步骤。

在这种情况下，我们可能会添加关于电视的额外信息，而不同情况下，我们可能希望提供关闭账户的链接或类似的内容。

我们将在以后的视频中了解更多有关处理输入的不同方法。

在下一个视频中，我们将探讨更多评估输入的方法，特别是确保用户以负责任的方式使用系统的方法。